Importar librerias requeridas

In [4]:
import ee
import geemap

In [5]:
ee.Authenticate()
ee.Initialize(project='ee-rimartinezs')

In [21]:

# Crear el mapa
Map = geemap.Map()
point = ee.Geometry.Point([-70.1,-3.4])
Map.centerObject(point, 11)

# Colección Landsat 8 con bandas seleccionadas
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
        ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    )

# Filtrar imágenes por fecha y nubes
pre_image = landsat8 \
    .filterBounds(point) \
    .filterDate('2015-01-01', '2015-06-30') \
    .sort('CLOUD_COVER') \
    .first()

post_image = landsat8 \
    .filterBounds(point) \
    .filterDate('2020-06-01', '2020-06-30') \
    .sort('CLOUD_COVER') \
    .first()

# Visualización del cambio NBR
palette = [
    '011959', '0E365E', '1D5561', '3E6C55', '687B3E',
    '9B882E', 'D59448', 'F9A380', 'FDB7BD', 'FACCFA'
]

# Calcular NBR
nbr_pre = pre_image.normalizedDifference(['nir', 'swir2']).rename('nbr_pre')
nbr_post = post_image.normalizedDifference(['nir', 'swir2']).rename('nbr_post')

diff = nbr_post.subtract(nbr_pre).rename('change')

vis_params = {
    'min': -0.2,
    'max': 0.2,
    'palette': palette
}
Map.addLayer(diff, vis_params, 'NBR Change')

# Clasificación según umbrales
threshold_gain = 0.10
threshold_loss = -0.10

diff_classified = ee.Image(0) \
    .where(diff.lte(threshold_loss), 2) \
    .where(diff.gte(threshold_gain), 1)

# Visualización de la clasificación
change_vis = {
    'min': 0,
    'max': 2,
    'palette': ['fcffc8', '2659eb', 'fa1373']
}
Map.addLayer(diff_classified.selfMask(), change_vis, 'Change Classified')

# Mostrar mapa
Map

Map(center=[-3.3999999999999995, -70.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…